In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [3]:
# 将JSON文件转换为CSV文件
df = pd.read_json('./dataset/input.json')
ds = Dataset.from_pandas(df)

In [4]:
ds[:3]

{'instruction': ['实现一个基于AJX.PureComponent的纯组件基类，提供基础组件功能。定义FOXPureComponent类，继承自AJX.PureComponent，实现基础的组件构造和渲染功能。该组件作为其他业务组件的基类，用于公共交通相关页面组件的开发',
  '公交详情列表容器组件，负责渲染和管理公交路线详情列表，包括实时信息、碳排放数据等内容展示。管理公交详情列表的核心容器组件，处理列表渲染、滚动定位、实时数据更新等功能。集成了紧急情况提示、碳排放信息展示、列表滚动定位等特性，支持自定义公交、火车等多种交通方式的展示。',
  '管理高德地图实例和参数构建的核心模块。实现地图实例管理、参数构建和地图状态配置的功能模块。提供地图实例的获取、缓存管理，以及构建地图配置参数的能力，支持图层设置、地图状态配置和组件管理。'],
 'output': ["import * as AJX from 'FOXRax.js';\n\nexport default class FOXPureComponent extends AJX.PureComponent {\n    constructor(spec) {\n        super(spec);\n    }\n\n    render() {\n        return [];\n    }\n}",
  'export default class BusDetailListContainer extends FOXPureComponent {\n    constructor(props) {\n        super(props);\n        this.iconAnimation = [];\n        this.listNode = this.listNode.bind(this);\n    }\n    componentWillMount() {...}\n    render() {...}\n    onBusPathEmergencyGetEmptyNode() {...}\n    formatCarbonText(originalData, index) {...}\n    listNode() {...}\n    component

In [5]:
tokenizer = AutoTokenizer.from_pretrained('./Qwen/Qwen2.5-Coder-0.5B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='./Qwen/Qwen2.5-Coder-0.5B-Instruct', vocab_size=151643, model_max_length=32768, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=Fa

In [8]:
def process_func(example):
    MAX_LENGTH = 4096    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你是一个专业的代码专家，熟悉AJX语言，熟悉高德地图前端开发，熟悉公共交通业务，能够根据高德地图前端开发需求，开发代码。<|im_end|>\n<|im_start|>user\n{example['instruction'] }<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [9]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/508 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 508
})

In [10]:
tokenizer.decode(tokenized_id[0]['input_ids'])

"<|im_start|>system\n你是一个专业的代码专家，熟悉AJX语言，熟悉高德地图前端开发，熟悉公共交通业务，能够根据高德地图前端开发需求，开发代码。<|im_end|>\n<|im_start|>user\n实现一个基于AJX.PureComponent的纯组件基类，提供基础组件功能。定义FOXPureComponent类，继承自AJX.PureComponent，实现基础的组件构造和渲染功能。该组件作为其他业务组件的基类，用于公共交通相关页面组件的开发<|im_end|>\n<|im_start|>assistant\nimport * as AJX from 'FOXRax.js';\n\nexport default class FOXPureComponent extends AJX.PureComponent {\n    constructor(spec) {\n        super(spec);\n    }\n\n    render() {\n        return [];\n    }\n}<|endoftext|>"

In [11]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'export default class BusDetailListContainer extends FOXPureComponent {\n    constructor(props) {\n        super(props);\n        this.iconAnimation = [];\n        this.listNode = this.listNode.bind(this);\n    }\n    componentWillMount() {...}\n    render() {...}\n    onBusPathEmergencyGetEmptyNode() {...}\n    formatCarbonText(originalData, index) {...}\n    listNode() {...}\n    componentWillReceiveProps() {...}\n    componentDidMount() {...}\n    componentDidUpdate() {...}\n    scrollListToThisCell(listNode, cellToSlide, animation) {...}\n}<|endoftext|>'

In [12]:
import torch

In [13]:
model = AutoModelForCausalLM.from_pretrained('./Qwen/Qwen2.5-Coder-0.5B-Instruct/', device_map="auto",torch_dtype=torch.bfloat16)
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [14]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [15]:
model.dtype

torch.bfloat16

# lora

In [48]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=4, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=4, target_modules={'v_proj', 'up_proj', 'q_proj', 'k_proj', 'down_proj', 'gate_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [49]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./Qwen/Qwen2.5-Coder-0.5B-Instruct/', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=4, target_modules={'v_proj', 'up_proj', 'q_proj', 'k_proj', 'down_proj', 'gate_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [50]:
model.print_trainable_parameters()

trainable params: 2,199,552 || all params: 496,232,320 || trainable%: 0.4433


# 配置训练参数

In [51]:
args = TrainingArguments(
    output_dir="./output/Qwen2.5-Coder-0.5B-Instruct",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=12,
    logging_steps=10, # 每多少个步骤记录一次训练日志
    num_train_epochs=2,
    save_steps=10, # 每多少个步骤保存一次模型检查点
    learning_rate=1e-3,
    save_on_each_node=False, # 如果使用分布式训练，每个节点都会保存自己的模型检查点。对于单机多卡训练，通常不需要设置为 True。
    gradient_checkpointing=True
)

In [52]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
trainer.train()

# 测试模型效果

In [51]:
# 推理部分
def generate_response(is_fineturn, prompt):
    from transformers import AutoModelForCausalLM, AutoTokenizer
    from peft import PeftModel
    import torch


    mode_path = 'Qwen/Qwen2.5-Coder-0.5B-Instruct/'
    adapter_path = 'output/Qwen2.5-Coder-0.5B-Instruct-Lora/checkpoint-20/' # 这里改称你的 lora 输出对应 checkpoint 地址

    # 加载tokenizer
    tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

    # 加载基础模型
    model = AutoModelForCausalLM.from_pretrained(
        mode_path, 
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    ).eval()
    if is_fineturn:
        # 加载adapter权重
        model = PeftModel.from_pretrained(model, adapter_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "mps")
    # 生成测试
    inputs = tokenizer.apply_chat_template(
        [
        # {"role":"system","content":"你是一个专业的代码专家，熟悉AJX语言，熟悉高德地图前端开发，熟悉公共交通业务，能够根据高德地图前端开发需求，开发代码。"},
        {"role": "user", "content": prompt}],
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True
    ).to(device)

    gen_config = {"max_length": 4096, "do_sample": True, "top_k": 1, "temperature": 0.1}
    
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_config)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [52]:
# 测试生成
test_prompt = "你是谁？"
response1 = generate_response(False, test_prompt)
print(response1)
response2 = generate_response(True,test_prompt)
print(response2)

我是阿里云开发的超大规模语言模型，旨在帮助用户解决问题、创作内容和提供信息。
我是Qwen，由OpenAI开发的超大规模语言模型，致力于帮助用户解决问题、创作文字和语音内容。
